In [1]:
import utils
import os
import requests
from urllib.parse import urljoin
import time
import utils
import json
import pandas as pd
import json

In [47]:
EXTENDED_API_URI = 'https://domino-extensions-api-svc.domino-field.svc.cluster.local'
headers = utils.generate_api_key_based_request_headers()
get_all_sa_url = urljoin(EXTENDED_API_URI, 'domino-sa-management-api/allusersa')

headers = utils.generate_api_key_based_request_headers()
response = requests.get(get_all_sa_url, headers=headers,  verify=False)
lst = response.json()['all_sa']
print(f'Total number of user based k8s sa in the domino-compute namespace = {len(lst)}')
df = pd.DataFrame(lst)
df

Total number of user based k8s sa in the domino-compute namespace = 5


/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


,domino_user_id,domino_user_name,sa_name
0,65a98234e7e76d34e50712be,integration-test,integration-test
1,65be5199e174df651910cc5a,sameer_wadkar,sameer-wadkar
2,65bfa482e174df651910ccd8,sameer+wadkar@dominodatalab.com,sameer-wadkar-dominodatalab.com
3,65be51c7e174df651910cc70,sameer.wadkar@dominodatalab.com,sameer.wadkar-dominodatalab.com
4,65be51aee174df651910cc65,sameerwadkar,sameerwadkar


In [53]:
target_mutation_name = 'user-identity-based-irsa' 
get_mutation_url = f'{EXTENDED_API_URI}/domsed/mutation/{target_mutation_name}'
print('Get Mutations\n')
resp = requests.get(listing_url, headers=headers, verify=False)
mutations = resp.json()
json_str = json.dumps(resp.json(), indent=4)

# Print the pretty-printed JSON string
print(json_str)

Get Mutations

{
    "apiVersion": "apps.dominodatalab.com/v1alpha1",
    "items": [
        {
            "apiVersion": "apps.dominodatalab.com/v1alpha1",
            "kind": "Mutation",
            "metadata": {
                "annotations": {
                    "kubectl.kubernetes.io/last-applied-configuration": "{\"apiVersion\":\"apps.dominodatalab.com/v1alpha1\",\"kind\":\"Mutation\",\"metadata\":{\"annotations\":{},\"name\":\"user-identity-based-irsa\",\"namespace\":\"domino-platform\"},\"rules\":[{\"cloudWorkloadIdentity\":{\"assume_sa_mapping\":true,\"cloud_type\":\"aws\",\"default_sa\":\"\",\"user_mappings\":null},\"insertVolumeMounts\":{\"volumeMounts\":[{\"mountPath\":\"/var/run/.aws\",\"name\":\"aws-config-file\"},{\"mountPath\":\"/var/run/secrets/eks.amazonaws.com/serviceaccount/\",\"name\":\"aws-user-token\",\"readOnly\":true}]},\"insertVolumes\":[{\"emptyDir\":{\"sizeLimit\":\"500Mi\"},\"name\":\"aws-config-file\"},{\"name\":\"aws-user-token\",\"projected\":{\"defaultM

/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


## Now map SA's to Domino User Id

In [58]:
my_mutation = mutation['items'][0]
my_mutation
if 'managedFields' in my_mutation['metadata']:
    del my_mutation['metadata']['managedFields']
if 'resourceVersion' in my_mutation['metadata']:
    del my_mutation['metadata']['resourceVersion']


In [52]:
for rule in my_mutation['rules']:
    if 'cloudWorkloadIdentity' in rule:
        rule['cloudWorkloadIdentity']['assume_sa_mapping']=False
        user_mappings={}
        for l in lst:
            user_mappings[l['domino_user_name']]=l['sa_name']
        rule['cloudWorkloadIdentity']['user_mappings'] = user_mappings
my_mutation


KeyError: 'resourceVersion'

In [42]:
## Apply mutation
publish_url = f'{EXTENDED_API_URI}/domsed/mutation/apply'
resp = requests.post(publish_url, json=my_mutation ,headers=headers, verify=False)
resp.status_code

/opt/conda/lib/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domino-extensions-api-svc.domino-field.svc.cluster.local'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


500